In [364]:
import pyodbc
import os
import logging 
import random
from datetime import datetime, timedelta


logging.basicConfig(format='%(levelname)s: %(asctime)s => %(message)s', level=logging.INFO)

connstr = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=nas.home.michaelfudge.com,11433;DATABASE=master;UID=sa;PWD=wXp55!929ds'

def runSql(sql):
    with pyodbc.connect(connstr, autocommit=True) as conn:
        commands = sql.split("GO\n")
        for command in commands:
            conn.execute(command)

def getCursor():
    with pyodbc.connect(connstr, autocommit=False) as conn:
        return conn.cursor()

        
def readFileContents(filespec):
    with open (filespec,'r') as f:
        return f.read()

In [360]:
def initializeDb(drop=True):
    script_path = 'sql'
    files = os.listdir(script_path)
    downscript = [file for file in files if file.startswith("dn-0")][0]
    if drop:
        print(f"FILE {downscript}")
        sql = readFileContents(os.path.join(script_path, downscript))
        print(sql)
        runSql(sql)
    upscripts = [file for file in files if file.startswith("up-")]
    upscripts.sort()
    for script in upscripts:
        sql = readFileContents(os.path.join(script_path, script))
        print(f"FILE {script}")
        print(sql)
        runSql(sql)
        
        

In [352]:
'''
    End date of None implies current date
'''
#def generateData(settings, start_date, end_date=None):
settings = [
    'churn' : {
        'trigger' : "daily"
        'execute': { 'times' : 1},
        'probability' : 0.005,
        'constraints' : {
            'min_threshold': 4,
            'jobtitles' : ['Sales Associate'],
            'jobtitles_weights' : [1],
            'departments' : [ 'Clothing', 'Customer Service', 'Electronics', 'Hardware', 'Housewares', 'Sporting Goods', 'Toys' ],
            'departments_weights' : [ 1, 4, 1, 1, 1, 1, 2]            
        }
    },
    'hires' : {
        'trigger' : "daily"
        'execute' : { 'times': 1 },
        'probability' : 0.01,
        'constraints': {
            'jobtitles' : ['Sales Associate'],
            'jobtitles_weights' : [1],
            'departments' : [ 'Clothing', 'Customer Service', 'Electronics', 'Hardware', 'Housewares', 'Sporting Goods', 'Toys' ],            
            'departments_weights' : [ 1, 2, 2, 1, 1, 2, 2]
        }
    },
    'transfers' : {
        'trigger' : "daily",
        'execute':  { 'times': 1 },
        'probability' : 0.01,
        'constraints' : {
            'min_threshold': 4,
            'jobtitles' : ['Sales Associate'],
            'jobtitles_weights' : [1],
            'departments' : [ 'Clothing', 'Customer Service', 'Electronics', 'Hardware', 'Housewares', 'Sporting Goods', 'Toys' ],
            'departments_weights' : [ 1, 1, 1, 1, 1, 1, 1]            
        }
    },
    'nov-transfers' {
        'trigger' : '11/20',
        'execute':  { 'times': 2 },
        'probability' : 0.9,
        'constraints' : {
            'min_threshold': 4,
            'jobtitles' : ['Sales Associate'],
            'jobtitles_weights' : [1],
            'from_departments' : [ 'Customer Service'],
            'from_departments_weights' : [ 1 ]
            'to_departments' : [ 'Electronics','Toys' ],
            'to_departments_weights' : [ 1, 1]            
        }
    },
    'jan-transfers' {
        'trigger' : '01/10',
        'execute':  { 'times': 1 },
        'probability' : 0.9,
        'constraints' : {
            'min_threshold': 4,
            'probability' : 0.9,
            'jobtitles' : ['Sales Associate'],
            'jobtitles_weights' : [1],
            'to_departments' : [ 'Customer Service'],
            'to_departments_weights' : [ 1 ]
            'from_departments' : [ 'Electronics','Toys' ],
            'from_departments_weights' : [ 1, 1]            
        }
    },
    'time-cards' : {
        'trigger' : "daily",
        'execute':  { 'foreach': 'Sales Associate' },
        'probability' : 0.75,
        'constraints' : {
            'departments' : [ 
                {'name' :'Clothing',             'distribution' : {
                'base' : 2,
                'offset': 1,
                'sample': 7               
            }

                {'name' :'Customer Service', 
                {'name' :'Electronics', 
                {'name' :'Hardware', 
                {'name' :'Housewares', 
                {'name' :'Sporting Goods', 
                {'name' :'Toys' 
            ],

            
            'departments
            'distribution' : {
                'base' : 2,
                'offset': 1,
                'sample': 7               
            }
        }
        
    }
    
    'raises' : {
            'base_pct' : 0.01,
            'jobtitles'
    }
}
start_date = '2018-01-05'
end_date = None

start_date = datetime.strptime(start_date, '%Y-%m-%d').date()
end_date = datetime.today().date()
start_date,end_date
    
    

(datetime.date(2018, 1, 5), datetime.date(2020, 3, 31))

In [345]:
end_date.month

3

In [10]:
import random
import math

'''
    Simulate a normal distribution of values through a dice simulator
'''
def roll_dice(random, number_of_rolls= 6, sides_on_die=6, start_at=0):
    total = 0
    for i in range(number_of_rolls):
        total += random.randint(start_at,sides_on_die-1+start_at)
    return total

'''
    Input the number of buckets and the whether you want a wide or narrow distribution and you 
    will get back the required arguments to roll dice
'''
def gen_dice(max_buckets, widest_normal_dist=True):
    n = max_buckets - 1
    if widest_normal_dist==True:
        for i in range(int(math.sqrt(n)),1,-1):
            if n%i == 0:
                break        
    else:
        for i in range(2,n+1):
            if n%i == 0:
                break

    rolls = n//i
    sides = i+1
    return rolls, sides

'''
    Get a random number between 0 and n, using a standard normal disribution.
    widest_normal_dist=True will use the widest distribuion possible (larger std)
'''
def rand_normal_dist(random, n, widest_normal_dist=True):
    rolls,sides = gen_dice(n,widest_normal_dist);
    return roll_dice(random,rolls, sides,0)



data = {}
for i in range(100000):
    n = rand_normal_dist(random, 9, widest_normal_dist=True)
    data[n] = data.get(n,0) + 1

keys = list(data.keys())
keys.sort()
    
for i in keys:
    print(f"{i}\t{data[i]}\t{2+i}")



0	1228	2
1	4878	3
2	12393	4
3	19788	5
4	23412	6
5	19837	7
6	12227	8
7	4914	9
8	1323	10


In [362]:
def add_time_entry(employee_id, payperiod_id, entry_date, hours_worked):
    pass

In [216]:
#TODO: Load in the lookup tables


In [169]:
#TODO: Create employees data generator
# setup rules 1 owner, 1-3 store managers, 1 department manager per department, 
# customer service  bell curve mean is 10 employees,
#churn rate, aquisition rate, (only amonng hourly)



In [368]:
initializeDb()
#generateData('2018-01-05')

FILE dn-0-drop-database.sql
USE [master]
GO
if exists (select name from sys.databases where name='payroll')
    ALTER DATABASE  payroll SET SINGLE_USER WITH ROLLBACK IMMEDIATE
GO
drop database if exists payroll;

FILE up-0-create-database.sql
create database payroll;


FILE up-1-create-schema.sql
use payroll;
/*
+-----------+         +---------+
| Employees |--------<|Paychecks|
+-----------+         +---------+
     |                    \|/
     |                     | 
    /|\                    |
+----------------+     +-----------+
|Timecard_Entries|>----|Pay_Periods|
+----------------+     +-----------+

*/ 


create table jobtitle_lookup
(
    jobtitle varchar(100) not null,
    payroll_type varchar(10) not null,
    constraint pk_jobtitle_lookup_jobtitle primary key (jobtitle)
);

create table department_lookup
(
    department varchar(100) not null,
    constraint pk_department_lookup_department primary key(department)
);

create table employees
(
    employee_id int not null,


In [36]:
downscript = [file for file in files if file.startswith("dn-0")]
downscript

['dn-0-drop-database.sql']

In [89]:
logging.basicConfig(format='%(asctime)s %(message)s')
logging.warning('is when this event was logged.')

In [284]:
int(math.sqrt(20))

4

In [287]:
list(range(4,1,-1))

[4, 3, 2]

In [314]:
x = []
for i in range(100):
    x.append(random.gauss(10,2))
    
x.sort()
x

[4.404073054824767,
 4.758661022679748,
 5.0394015480885965,
 5.72565728447792,
 5.729688382805447,
 6.018283013140163,
 7.016180957960941,
 7.091014408426261,
 7.265261769299427,
 7.397881664250989,
 7.501834026484654,
 7.739692711345996,
 7.772170133207673,
 7.815087832335365,
 7.816913749858646,
 7.8498057635152865,
 8.21086473480613,
 8.230113236965906,
 8.246738297359768,
 8.617596484956366,
 8.843002034416504,
 8.963574332368816,
 9.030156872807648,
 9.109221287917212,
 9.15425092739899,
 9.169962548017772,
 9.219820885011032,
 9.441048321846875,
 9.467361226493784,
 9.57178856678434,
 9.575705503109736,
 9.594088167808748,
 9.59446729697515,
 9.680617019439005,
 9.72420008087835,
 9.790761228239955,
 9.8280086551102,
 9.830184557667163,
 9.871535733202556,
 9.879610250243076,
 9.968301898679337,
 10.013189501616152,
 10.072131582632908,
 10.109049947783001,
 10.125969516733582,
 10.135088470975148,
 10.150376632768236,
 10.162917313358014,
 10.280517477730013,
 10.28292317820742

In [12]:
string ="10d"
string[-1]

'd'

In [24]:
import time
x = time.strptime("2012-12-30 00:00", "%Y-%m-%d %H:%M")

In [23]:
string = "2012-12-30 00:00.1234"
string= "2010-09-14"
string[:string.find(".")]

''

In [28]:
def to_datetime_string(timestruct):
    t=timestruct
    return f"{t.tm_year:04}-{t.tm_mon:02}-{t.tm_mday:02} {t.tm_hour:02}:{t.tm_min:02}:{t.tm_sec:02}"
        
to_datetime_string(x)

'2012-12-30 00:00:00'

In [31]:
import datetime
datetime.datetime.


TypeError: an integer is required (got type str)